In [ ]:
%matplotlib inline
from SALib.sample import saltelli
# from wolf_sheep.model import WolfSheep
# from wolf_sheep.agents import Wolf, Sheep
from mesa.batchrunner import BatchRunner
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from model import GeoModel
from multiprocess import Pool
import os
%load_ext autoreload
%autoreload 2

## Sobol analyses

In [ ]:
# We define our variables and bounds
problem = {
    'num_vars': 5,
    'names': ['num_buildings', 'num_destinations', 'trace_strength', 'agent_vision_angle', 'agent_vision_samples'],
    'bounds': [[50,100], [3,10], [10,100], [7.5,90], [3,15]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 30
max_steps = 170
distinct_samples = 10

model_reporters = {'Avg_raster_value': lambda m: m.avg_raster_value}

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples)

# READ NOTE BELOW CODE
batch = BatchRunner(GeoModel,
                    max_steps=max_steps,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters)

count = 0
data = pd.DataFrame(index=range(replicates*len(param_values)), 
                                columns=['num_buildings', 'num_destinations', 'trace_strength', 'agents_vision_angle', 'agent_vision_samples'])
data['Run'], data['Avg_raster_value'] = None, None

rep_values = []
for i in range(replicates):
    rep_values.append(param_values)

In [ ]:
# define function for mulitprocessing
def vals_run(param_values):
    data = pd.DataFrame(index=range(len(param_values)), 
                                columns=['num_buildings', 'num_destinations', 'trace_strength', 'agents_vision_angle', 'agent_vision_samples'])
    data['Run'], data['Avg_raster_value'] = None, None

    count = 0
    for vals in param_values: 
            # Change parameters that should be integers
            vals = list(vals)
            vals[0] = int(vals[0])
            vals[1] = int(vals[1])
            vals[2] = int(vals[2])
            vals[4] = int(vals[4])
            # Transform to dict with parameter names and their values
            variable_parameters = {}
            for name, val in zip(problem['names'], vals):
                variable_parameters[name] = val

            batch.run_iteration(variable_parameters, tuple(vals), count)
            iteration_data = batch.get_model_vars_dataframe().iloc[count]
            iteration_data['Run'] = count # Don't know what causes this, but iteration number is not correctly filled
            data.iloc[count, 0:5] = vals
            data.iloc[count, 5:7] = iteration_data
            count += 1
    
    return data

In [ ]:
# run multiprocessing
if __name__ == '__main__':
    pool = Pool((os.cpu_count()))
    results = pool.map(vals_run, rep_values)
    pool.close

In [ ]:
# concat the list of dataframs into one and save it as .csv
final_data = pd.concat(results, ignore_index=True)
final_data.to_csv('data/sobol_results.csv')

## OFAT

In [ ]:
from mesa.batchrunner import BatchRunnerMP

In [ ]:
if __name__ == '__main__':
    # We define our variables and bounds
    problem = {
        'num_vars': 5,
        'names': ['num_buildings', 'num_destinations', 'trace_strength', 'agent_vision_angle', 'agent_vision_samples'],
        'bounds': [[50,100,11], [3,10,8], [10,100,10], [7.5,90,12], [3,15,7]]
        # 'bounds': [[50,100,2], [2,10,2], [10,100,2], [7.5,90,2], [3,15,2]]
    }

    # Set the repetitions, the amount of steps, and the amount of distinct values per variable
    replicates = 30
    max_steps = 170

    # Set the outputs
    model_reporters = {'Avg_raster_value': lambda m: m.avg_raster_value}

    data = pd.DataFrame()

    for i, var in enumerate(problem['names']):
        # Get the bounds for this variable and get <distinct_samples> samples within this space (uniform)
        samples = np.linspace(*problem['bounds'][i], dtype=int)
        
        # # Keep in mind that wolf_gain_from_food should be integers. You will have to change
        # # your code to acommodate for this or sample in such a way that you only get integers.
        if var == 'agent_vision_angle':
            samples = np.linspace(*problem['bounds'][i])
        
        # use BatchRunnerMP for multiprocessing
        batch = BatchRunnerMP(GeoModel, nr_processes = None,
                            max_steps=max_steps,
                            iterations=replicates,
                            variable_parameters={var: samples},
                            model_reporters=model_reporters,
                            display_progress=True)
        
        batch.run_all()
        
        data = pd.concat([data, batch.get_model_vars_dataframe()])
        data.to_csv('data/OFAT_data.csv')